# Data Collection

In [1]:
from keras.preprocessing.image import ImageDataGenerator

## Configure ImageDataGenerator class

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

## Importing the Dataset

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='I2NtiNmxn6RMB80AK1KzcqFmssPEs0muy-u9vVEfGjdF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasetoolforsterilebrowsin-donotdelete-pr-a9je6fiuqleyv6'
object_key = 'Dataset-IBM.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


## Unzipping the Dataset

In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [6]:
filenames = os.listdir('/home/wsuser/work/Dataset/train')

## Apply ImageDataGenerator functionality to Trainset and Testset

In [7]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/train/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


# Model building

## Importing the Model Building Libraries

In [8]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


## Initializing the Model

In [9]:
classifier = Sequential()

### Adding CNN Layers

In [10]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

### Adding Dense Layers

In [11]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))

In [12]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

## Configure The Learning Process

In [13]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

/tmp/wsuser/ipykernel_217/2617134232.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
119/119 [==============================] - 6s 43ms/step - loss: 1.3614 - accuracy: 0.4832 - val_loss: 0.6568 - val_accuracy: 0.7667
Epoch 2/20
119/119 [==============================] - 5s 42ms/step - loss: 0.6663 - accuracy: 0.7340 - val_loss: 0.5007 - val_accuracy: 0.9000
Epoch 3/20
119/119 [==============================] - 5s 42ms/step - loss: 0.4844 - accuracy: 0.8081 - val_loss: 0.5624 - val_accuracy: 0.8000
Epoch 4/20
119/119 [==============================] - 5s 41ms/step - loss: 0.3675 - accuracy: 0.8653 - val_loss: 0.4007 - val_accuracy: 0.8667
Epoch 5/20
119/119 [==============================] - 5s 42ms/step - loss: 0.3375 - accuracy: 0.8670 - val_loss: 0.3236 - val_accuracy: 0.9000
Epoch 6/20
119/119 [==============================] - 5s 42ms/step - loss: 0.2559 - accuracy: 0.9108 - val_loss: 0.3335 - val_accuracy: 0.9333
Epoch 7/20
119/119 [==============================] - 5s 40ms/step - loss: 0.2045 - accuracy: 0.9293 - val_loss: 0.3956 - val_accuracy: 0.9333

## Save the Model

In [15]:
classifier.save('gesture.h5')

In [16]:
!tar -zcvf gesture.tgz gesture.h5

gesture.h5


In [17]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

## Test the model

In [18]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [19]:
model = load_model("gesture.h5")

In [20]:
from tensorflow.keras.utils import load_img, img_to_array

img = load_img("/home/wsuser/work/Dataset/test/0/0.jpg", grayscale=True, target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)

pred

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


array([[0., 0., 0., 0., 0., 1.]], dtype=float32)

In [21]:
pred[0]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

## Model on IBM

In [22]:
ls -1

Dataset/
gesture.h5
gesture.tgz
model-bw.json


In [23]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 15.4 MB/s eta 0:00:01


In [24]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "p7aLI5JVOibkfAkIEf2ncv6XJ1GIN7Ti716fylzGWKwF"
}
client = APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client, 'A Gesture Base Tool for Sterile Browsing of Radiology Images - PNT2022TMID53035')
print("Space UID = "+space_uid)

Space UID = 011071d9-17ae-4f64-9330-0e40296d9285


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list(limit = 100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid 

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [30]:
model_details = client.repository.store_model(model="gesture.tgz",meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [31]:
model_id

'23961632-d07f-4901-ad61-0d36f1b10534'

In [32]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'